In [1]:
from __future__ import print_function
import sys
import os
import re, gzip, time, itertools, io
import sys
import pandas as pd
import numpy as np
from numpy import array, diff, where, split
import requests
import matplotlib.pyplot as plt
import metapredict as meta
import Bio
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from pandarallel import pandarallel
from itertools import chain

In [9]:
with open('Fasta/martquery_0622162206_107.fasta') as fasta_file:  # Will close handle cleanly
    identifiers = []
    length = []
    sequences = []
    for title, sequence in SimpleFastaParser(fasta_file):
        identifiers.append(title)
        length.append(len(sequence))
        sequences.append(sequence)
s1 = pd.Series(identifiers, name='PepID')
s2 = pd.Series(sequences, name='sequence')
s3 = pd.Series(length, name='length')
dfpep = pd.DataFrame(dict(PepID=s1, sequence=s2, length=s3))
dfpep[['GeneID', 'TrnID','GeneName', 'PepID', 'TrnsType', 'PepType']] = dfpep['PepID'].str.split('|', expand=True)
dfpep = dfpep[['PepID', 'sequence', 'length']]
dfpep.head()

,PepID,sequence,length
0,ENSP00000054650,MPKSCAARQCCNRYSSRRKQLTFHRFPFSRPELLKEWVLNIGRGNF...,239
1,ENSP00000005082,MTLLTFRDVAIEFSLEEWKCLDLAQQNLYRDVMLENYRNLFSVGLT...,606
2,ENSP00000020945,MPRSFLVKKHFNASKKPNYSELDTHTVIISPYLYESYSMPVIPQPE...,268
3,ENSP00000155093,MDEDEFELQPQEPNSFFDGIGADATHMDGDQIVVEIQEAVFVSNIV...,801
4,ENSP00000190165,MNGYGSPYLYMGGPVSQPPRAPLQRTPKCARCRNHGVLSWLKGHKR...,472


In [10]:
df = pd.read_csv('csv/TFs_canonical.csv',low_memory=False)
df.head()

,GeneID,TrnID,PepID,GeneName,GeneFamily,UniProtKB_ID,UniProtKB_Gene_Name
0,ENSG00000101126,ENST00000621696,ENSP00000483881,ADNP,Homeobox,Q9H2P0,ADNP_HUMAN
1,ENSG00000101544,ENST00000262198,ENSP00000262198,ADNP2,TF_others,Q6IQ32,ADNP2_HUMAN
2,ENSG00000139154,ENST00000266508,ENSP00000266508,AEBP2,zf-C2H2,Q6ZN18,AEBP2_HUMAN
3,ENSG00000172493,ENST00000395146,ENSP00000378578,AFF1,AF-4,P51825,AFF1_HUMAN
4,ENSG00000155966,ENST00000370460,ENSP00000359489,AFF2,AF-4,P51816,AFF2_HUMAN


In [11]:
df = pd.merge(df, dfpep, how="left", left_on="PepID", right_on="PepID")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1653 entries, 0 to 1652
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   GeneID               1653 non-null   object
 1   TrnID                1653 non-null   object
 2   PepID                1653 non-null   object
 3   GeneName             1638 non-null   object
 4   GeneFamily           1653 non-null   object
 5   UniProtKB_ID         1652 non-null   object
 6   UniProtKB_Gene_Name  1632 non-null   object
 7   sequence             1653 non-null   object
 8   length               1653 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 129.1+ KB


In [12]:
Human_curratedDB = pd.read_csv('Human_curratedDB.csv',low_memory=False)
Human_curratedDB = Human_curratedDB['ID'].tolist()
len(Human_curratedDB)

1417

In [13]:
df = df[df['GeneID'].isin(Human_curratedDB)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392 entries, 0 to 1636
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   GeneID               1392 non-null   object
 1   TrnID                1392 non-null   object
 2   PepID                1392 non-null   object
 3   GeneName             1392 non-null   object
 4   GeneFamily           1392 non-null   object
 5   UniProtKB_ID         1392 non-null   object
 6   UniProtKB_Gene_Name  1392 non-null   object
 7   sequence             1392 non-null   object
 8   length               1392 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 108.8+ KB


In [14]:
seq_id = []
genename = []
seq = []

output_path = 'Fasta/HumanTF_canonical.fasta'
output_file = open(output_path,'w')

for i, row in df.iterrows():
    seq_id = df['PepID'][i]
    genename = df['GeneName'][i]
    seq = df['sequence'][i]
    identifier_line = ">" + seq_id + " " + str(genename) + "\n"
    output_file.write(identifier_line)
    sequence_line = seq + "\n"
    output_file.write(sequence_line)
            
    
#Close the file when we're done
output_file.close()

In [15]:
with open('Fasta/Human_TF.IDR.fasta') as fasta_file:  # Will close handle cleanly
    identifiers = []
    length = []
    sequences = []
    for title, sequence in SimpleFastaParser(fasta_file):
        identifiers.append(title)
        length.append(len(sequence))
        sequences.append(sequence)
s1 = pd.Series(identifiers, name='ID')
s2 = pd.Series(sequences, name='sequence')
s3 = pd.Series(length, name='length')
TFIDR = pd.DataFrame(dict(ID=s1, IDRSeq=s2, IDRlength=s3))
p1 = re.compile(r'(?P<PepID>[^\s]+)+'
                   + r'.IDR_START=(?P<IDR_START>\d{1,5})'
                   + r'.IDR_END=(?P<IDR_END>\d{1,5})')
TFIDR = pd.concat([
    TFIDR,
    (
        TFIDR.ID.str.extractall(p1)
          .reset_index('match', drop=True)
    )
], axis=1).fillna('')
TFIDR = TFIDR.drop('ID', 1)

TFIDR = TFIDR[['PepID','IDRSeq', 'IDRlength',  'IDR_START', 'IDR_END']]

TFIDR = TFIDR[TFIDR['IDRlength'] >= 50]

TFIDR2 = TFIDR.groupby('PepID').apply(lambda x: [list(x['IDRSeq']), list(x['IDRlength']), list(x['IDR_START']), list(x['IDR_END'])]).apply(pd.Series).reset_index()

TFIDR2.columns =['PepID','IDRSeq', 'IDRlength',  'IDR_START', 'IDR_END']



TFIDR.tail()

/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_62816/2745064662.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  TFIDR = TFIDR.drop('ID', 1)


,PepID,IDRSeq,IDRlength,IDR_START,IDR_END
3578,ENSP00000353132,QLQKQKEAEGSHGEGKAEAPTPDTQLDTEQPPRLPGSDPPAELHLS...,246,131,377
3579,ENSP00000356674,LSQTRTKHQERPPESHRVTPNLNRSLSPRHNTPKGNRRGQVSAVLD...,212,137,349
3580,ENSP00000356674,PGCIPLEGPHSISPETTVTSRGQAEEESPSQEETVAPGEAVQGSVS...,51,452,503
3581,ENSP00000302189,MMLNSDTMELDLPPTHSETESGFSDCGGGAGPDGAGPGGPGGGQAR...,60,0,60
3582,ENSP00000351539,EKATTEADSTKKEEDASSCSDKVESLSDGSSHIAGDLPSDEDEGED...,291,127,418


In [16]:
df = pd.merge(df, TFIDR2, how="left", left_on="PepID", right_on="PepID")

In [18]:
df.to_csv('IDRS_humanTF.csv', index=False)

In [19]:
 #df_test = df[(df['GeneName'] == "HOXD4")]# | (df['GeneName'] == "EGR1")]
 #df_test.head()

In [20]:
df_test = df.dropna()

def count_fyw(x):
    seq = x['sequence']
    Aromatic_num = len(re.findall('[FYW]', seq))
    return Aromatic_num

df_test["Aromatic_num"] = df_test.apply(count_fyw, axis=1,)
df_test.info()

df_test = df_test[(df_test['Aromatic_num'] >= 4)]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 1391
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   GeneID               1341 non-null   object
 1   TrnID                1341 non-null   object
 2   PepID                1341 non-null   object
 3   GeneName             1341 non-null   object
 4   GeneFamily           1341 non-null   object
 5   UniProtKB_ID         1341 non-null   object
 6   UniProtKB_Gene_Name  1341 non-null   object
 7   sequence             1341 non-null   object
 8   length               1341 non-null   int64 
 9   IDRSeq               1341 non-null   object
 10  IDRlength            1341 non-null   object
 11  IDR_START            1341 non-null   object
 12  IDR_END              1341 non-null   object
 13  Aromatic_num         1341 non-null   int64 
dtypes: int64(2), object(12)
memory usage: 157.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1336 e

/var/folders/dd/83fsp94d6_98_nzr92wk_nt5nn0dv6/T/ipykernel_62816/1584201226.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Aromatic_num"] = df_test.apply(count_fyw, axis=1,)


In [30]:
def u(x, sep=','):
    x1 = [str(i) for i in x]
    return sep.join(x1)


OtherAA = "ARNDCEQGHILKMPSTV"

aa = 'FYW'


thres = 4


def get_dist(aa, seq):
    ## regular expression pattern to check for
    aacount = seq.count(aa)
    restr = '([' + aa + '])'
    cbeg = 0
    length = list()
    FYW = list()
    match = list()


    for m in re.finditer(restr, seq):
        if cbeg == 0 and m.start() == 0:
            cbeg = m.start()
            continue

        curdist = m.start() - cbeg - 1
        cbeg = m.start()
        cend = m.end()
        if curdist != -1:
            length.append(curdist)
            match.append(m.end())
            new_dict = dict(zip(match, length))

    return new_dict

def group_runs(keys, jmin, jmax):
    temp = []
    output = []
    for i in range(len(keys) - 1):
        j = i + 1
        if i == 0:
            h = 0
        else:
            h = i - 1

        key1 = keys[i]
        key2 = keys[j]
        key3 = keys[h]
        var1 = key2 - key1
        var2 = key1 - key3
        #if jmin <= var1 <= jmax:
        if jmin <= var1 <= jmax or var1 == 1 and var2 != 1:
            if key1 not in temp:
                temp.append(key1)
            temp.append(key2)
        else:
            if len(temp) > 0:
                output.append(temp)
            temp = []
    if len(temp) > 0:
        output.append(temp)
    #return output
    return [x for x in output if len(x) >= thres]


distD = dict()
distD['GeneName'] = list()
distD['GeneID'] = list()
distD['TrnID'] = list()
distD['PepID'] = list()
distD['UniProtKB_ID'] = list()
distD['UniProtKB_Gene_Name'] = list()
distD['GeneFamily'] = list()
distD['spacer_type'] = list()
distD['len'] = list()
distD['FYW'] = list()
distD['match'] = list()
distD['coordinates'] = list()
distD['sd'] = list()
distD['mean'] = list()
distD['median'] = list()
distD['cv'] = list()
distD['in_IDR'] = list()
distD['IDR_num'] = list()
distD['IDR_match_seq'] = list()
distD['4aaBlock_match_seq'] = list()
distD['8aaBlock_match_seq'] = list()
distD['2aaBlock_match_seq'] = list()

for i, row in df_test.iterrows():

    name = df_test['GeneName'][i]
    GeneID = df_test['GeneID'][i]
    TrnID = df_test['TrnID'][i]
    PepID = df_test['PepID'][i]
    UniProtKB_ID = df_test['UniProtKB_ID'][i]
    UniProtKB_Gene_Name = df_test['UniProtKB_Gene_Name'][i]
    seq = str(df_test['sequence'][i])


    seqA = re.sub('[ARNDCEQGHILKMPSTV]', '.', seq)
    GeneFamily = df_test['GeneFamily'][i]
    idr_s = df_test['IDR_START'][i]
    idr_e = df_test['IDR_END'][i]

    IDR_coordinates = []

    for n in range(len(idr_s)):
        IDR_coordinates.append(pd.Interval(int(idr_s[n]), int(idr_e[n]), closed='both'))

    arrayIDR = pd.arrays.IntervalArray(IDR_coordinates)
    
    # make a dic of aromatic positions and distance to a nearest aromatic
    dis_dic = get_dist(aa, seqA)
    dis_dic = {key: value for key, value in dis_dic.items() if np.any(arrayIDR.contains(key) == True) }

    # extract keys
    keys = list(dis_dic.keys())

    # filter keys
    #res0_3 = group_runs(keys, 0, 3)
    res4_10 = group_runs(keys, 4, 10)
    res11_20 = group_runs(keys, 11, 20)
    res21_30 = group_runs(keys, 21, 30)

    #final_res0_3 = []
    final_res4_10 = []
    final_res11_20 = []
    final_res21_30 = []

    #spacers_res0_3 = []
    spacers_res4_10 = []
    spacers_res11_20 = []
    spacers_res21_30 = []

    # for i in range(len(res0_3)):
    #     final_res0_3.append(dict(((key, dis_dic[key]) for key in res0_3[i])))

    for i in range(len(res4_10)):
        final_res4_10.append(dict(((key, dis_dic[key]) for key in res4_10[i])))
        
    for i in range(len(res11_20)):
        final_res11_20.append(dict(((key, dis_dic[key]) for key in res11_20[i])))
        
    for i in range(len(res21_30)):
        final_res21_30.append(dict(((key, dis_dic[key]) for key in res21_30[i])))

    # for i in range(len(final_res0_3)):
    #     spacers_res0_3.append(list(final_res0_3[i].values()))

    for i in range(len(final_res4_10)):
        spacers_res4_10.append(list(final_res4_10[i].values()))
        
    for i in range(len(final_res11_20)):
        spacers_res11_20.append(list(final_res11_20[i].values()))
        
    for i in range(len(final_res21_30)):
        spacers_res21_30.append(list(final_res21_30[i].values()))

    # for j in range(len(res0_3)):
    #
    #     d = dict()
    #
    #     spacer_type = "0_3"
    #
    #     coordinates = pd.Interval(res0_3[j][0], res0_3[j][-1], closed='both')
    #     mstart = res0_3[j][0] - 1
    #     mend = res0_3[j][-1]
    #     testIDR = arrayIDR.overlaps(coordinates)
    #     testIDR_num = np.where(testIDR == True)
    #
    #     d = pd.Series(spacers_res0_3[j])
    #     distD['GeneName'].append(name)
    #     distD['GeneID'].append(GeneID)
    #     distD['TrnID'].append(TrnID)
    #     distD['PepID'].append(PepID)
    #     distD['GeneFamily'].append(GeneFamily)
    #     distD['UniProtKB_ID'].append(UniProtKB_ID)
    #     distD['UniProtKB_Gene_Name'].append(UniProtKB_Gene_Name)
    #     distD['len'].append(spacers_res0_3[j])
    #     distD['spacer_type'].append(spacer_type)
    #     distD['FYW'].append(len(res0_3[j]))
    #     distD['coordinates'].append([int(mstart), int(mend)])
    #     distD['match'].append(seqA[mstart:mend])
    #     # distD['spacer']= spacer
    #     # distD['Pep_lenght']= len(seq)
    #     distD['sd'].append(d.std())
    #     distD['mean'].append(d.mean())
    #     distD['median'].append(d.median())
    #     if d.mean() and d.std():
    #         cv = d.std() / d.mean()
    #     distD['cv'].append(cv)
    #     distD['in_IDR'].append(np.any(testIDR == True))
    #     distD['IDR_num'].append(*testIDR_num[0])
    #     if mstart < 4:
    #         distD['IDR_match_seq'].append(seq[0:mend+4])
    #     else :
    #         distD['IDR_match_seq'].append(seq[mstart-4:mend+4])
    #     List4aaBlock_match_seq = list()
    #     List8aaBlock_match_seq = list()
    #     for w in range(len(res0_3[j])):
    #         if mstart < 4 and res0_3[j][w] == 1 :
    #             List4aaBlock_match_seq.append(seq[0:res0_3[j][w]+4])
    #         else:
    #             List4aaBlock_match_seq.append(seq[res0_3[j][w]-4:res0_3[j][w]+4])
    #     distD['4aaBlock_match_seq'].append(List4aaBlock_match_seq)
    #     for w in range(len(res0_3[j])):
    #         if mstart < 8 and res0_3[j][w] == 1 :
    #             List8aaBlock_match_seq.append(seq[0:res0_3[j][w]+8])
    #         else:
    #             List8aaBlock_match_seq.append(seq[res0_3[j][w]-8:res0_3[j][w]+8])
    #     distD['8aaBlock_match_seq'].append(List8aaBlock_match_seq)


    for j in range(len(res4_10)):

        d = dict()
    
        spacer_type = "4_10"
    
        coordinates = pd.Interval(res4_10[j][0], res4_10[j][-1], closed='both')
        mstart = res4_10[j][0] - 1
        mend = res4_10[j][-1]
        testIDR = arrayIDR.overlaps(coordinates)
        testIDR_num = np.where(testIDR == True)
    
        d = pd.Series(spacers_res4_10[j])
        distD['GeneName'].append(name)
        distD['GeneID'].append(GeneID)
        distD['TrnID'].append(TrnID)
        distD['PepID'].append(PepID)
        distD['GeneFamily'].append(GeneFamily)
        distD['UniProtKB_ID'].append(UniProtKB_ID)
        distD['UniProtKB_Gene_Name'].append(UniProtKB_Gene_Name)
        distD['len'].append(spacers_res4_10[j])
        distD['spacer_type'].append(spacer_type)
        distD['FYW'].append(len(res4_10[j]))
        distD['coordinates'].append([int(mstart), int(mend)])
        distD['match'].append(seqA[mstart:mend])
        # distD['spacer']= spacer
        # distD['Pep_lenght']= len(seq)
        distD['sd'].append(d.std())
        distD['mean'].append(d.mean())
        distD['median'].append(d.median())
        if d.mean() and d.std():
            cv = d.std() / d.mean()
        distD['cv'].append(cv)
        distD['in_IDR'].append(np.any(testIDR == True))
        distD['IDR_num'].append(*testIDR_num[0])
        if mstart < 4:
            distD['IDR_match_seq'].append(seq[0:mend+4])
        else :
            distD['IDR_match_seq'].append(seq[mstart-4:mend+4])
        List4aaBlock_match_seq = list()
        List8aaBlock_match_seq = list()
        List2aaBlock_match_seq = list()
        for w in range(len(res4_10[j])):
            if mstart < 4 and res4_10[j][w] == 1 :
                List4aaBlock_match_seq.append(seq[0:res4_10[j][w]+4])
            else:
                List4aaBlock_match_seq.append(seq[res4_10[j][w]-4:res4_10[j][w]+4])
        distD['4aaBlock_match_seq'].append(List4aaBlock_match_seq)
        for w in range(len(res4_10[j])):
            if mstart < 8 and res4_10[j][w] == 1 :
                List8aaBlock_match_seq.append(seq[0:res4_10[j][w]+8])
            else:
                List8aaBlock_match_seq.append(seq[res4_10[j][w]-8:res4_10[j][w]+8])
        distD['8aaBlock_match_seq'].append(List8aaBlock_match_seq)
        for w in range(len(res4_10[j])):
            if mstart < 2 and res4_10[j][w] == 1 :
                List2aaBlock_match_seq.append(seq[0:res4_10[j][w]+2])
            else:
                List2aaBlock_match_seq.append(seq[res4_10[j][w]-3:res4_10[j][w]+2])
        distD['2aaBlock_match_seq'].append(List2aaBlock_match_seq)

    for j in range(len(res11_20)):

        d = dict()

        spacer_type = "11_20"

        coordinates = pd.Interval(res11_20[j][0], res11_20[j][-1], closed='both')
        mstart = res11_20[j][0] - 1
        mend = res11_20[j][-1]
        testIDR = arrayIDR.overlaps(coordinates)
        testIDR_num = np.where(testIDR == True)

        d = pd.Series(spacers_res11_20[j])
        distD['GeneName'].append(name)
        distD['GeneID'].append(GeneID)
        distD['TrnID'].append(TrnID)
        distD['PepID'].append(PepID)
        distD['GeneFamily'].append(GeneFamily)
        distD['UniProtKB_ID'].append(UniProtKB_ID)
        distD['UniProtKB_Gene_Name'].append(UniProtKB_Gene_Name)
        distD['len'].append(spacers_res11_20[j])
        distD['spacer_type'].append(spacer_type)
        distD['FYW'].append(len(res11_20[j]))
        distD['coordinates'].append([int(mstart), int(mend)])
        distD['match'].append(seqA[mstart:mend])
        # distD['spacer']= spacer
        # distD['Pep_lenght']= len(seq)
        distD['sd'].append(d.std())
        distD['mean'].append(d.mean())
        distD['median'].append(d.median())
        if d.mean() and d.std():
            cv = d.std() / d.mean()
        distD['cv'].append(cv)
        distD['in_IDR'].append(np.any(testIDR == True))
        distD['IDR_num'].append(*testIDR_num[0])
        if mstart < 4:
            distD['IDR_match_seq'].append(seq[0:mend+4])
        else :
            distD['IDR_match_seq'].append(seq[mstart-4:mend+4])
        List4aaBlock_match_seq = list()
        List8aaBlock_match_seq = list()
        List2aaBlock_match_seq = list()
        for w in range(len(res11_20[j])):
            if mstart < 4 and res11_20[j][w] == 1 :
                List4aaBlock_match_seq.append(seq[0:res11_20[j][w]+4])
            else:
                List4aaBlock_match_seq.append(seq[res11_20[j][w]-4:res11_20[j][w]+4])
        distD['4aaBlock_match_seq'].append(List4aaBlock_match_seq)
        for w in range(len(res11_20[j])):
            if mstart < 8 and res11_20[j][w] == 1 :
                List8aaBlock_match_seq.append(seq[0:res11_20[j][w]+8])
            else:
                List8aaBlock_match_seq.append(seq[res11_20[j][w]-8:res11_20[j][w]+8])
        distD['8aaBlock_match_seq'].append(List8aaBlock_match_seq)
        for w in range(len(res11_20[j])):
            if mstart < 2 and res11_20[j][w] == 1 :
                List2aaBlock_match_seq.append(seq[0:res11_20[j][w]+2])
            else:
                List2aaBlock_match_seq.append(seq[res11_20[j][w]-3:res11_20[j][w]+2])
        distD['2aaBlock_match_seq'].append(List2aaBlock_match_seq)

    for j in range(len(res21_30)):

        d = dict()

        spacer_type = "21_30"

        coordinates = pd.Interval(res21_30[j][0], res21_30[j][-1], closed='both')
        mstart = res21_30[j][0] - 1
        mend = res21_30[j][-1]
        testIDR = arrayIDR.overlaps(coordinates)
        testIDR_num = np.where(testIDR == True)

        d = pd.Series(spacers_res21_30[j])
        distD['GeneName'].append(name)
        distD['GeneID'].append(GeneID)
        distD['TrnID'].append(TrnID)
        distD['PepID'].append(PepID)
        distD['GeneFamily'].append(GeneFamily)
        distD['UniProtKB_ID'].append(UniProtKB_ID)
        distD['UniProtKB_Gene_Name'].append(UniProtKB_Gene_Name)
        distD['len'].append(spacers_res21_30[j])
        distD['spacer_type'].append(spacer_type)
        distD['FYW'].append(len(res21_30[j]))
        distD['coordinates'].append([int(mstart), int(mend)])
        distD['match'].append(seqA[mstart:mend])

        # distD['spacer']= spacer
        # distD['Pep_lenght']= len(seq)
        distD['sd'].append(d.std())
        distD['mean'].append(d.mean())
        distD['median'].append(d.median())
        if d.mean() and d.std():
            cv = d.std() / d.mean()
        distD['cv'].append(cv)
        distD['in_IDR'].append(np.any(testIDR == True))
        distD['IDR_num'].append(*testIDR_num[0])
        if mstart < 4:
            distD['IDR_match_seq'].append(seq[0:mend+4])
        else :
            distD['IDR_match_seq'].append(seq[mstart-4:mend+4])
        List4aaBlock_match_seq = list()
        List8aaBlock_match_seq = list()
        List2aaBlock_match_seq = list()
        for w in range(len(res21_30[j])):
            if mstart < 4 and res21_30[j][w] == 1 :
                List4aaBlock_match_seq.append(seq[0:res21_30[j][w]+4])
            else:
                List4aaBlock_match_seq.append(seq[res21_30[j][w]-4:res21_30[j][w]+4])
        distD['4aaBlock_match_seq'].append(List4aaBlock_match_seq)
        for w in range(len(res21_30[j])):
            if mstart < 8 and res21_30[j][w] == 1 :
                List8aaBlock_match_seq.append(seq[0:res21_30[j][w]+8])
            else:
                List8aaBlock_match_seq.append(seq[res21_30[j][w]-8:res21_30[j][w]+8])
        distD['8aaBlock_match_seq'].append(List8aaBlock_match_seq)
        for w in range(len(res21_30[j])):
            if mstart < 2 and res21_30[j][w] == 1 :
                List2aaBlock_match_seq.append(seq[0:res21_30[j][w]+2])
            else:
                List2aaBlock_match_seq.append(seq[res21_30[j][w]-3:res21_30[j][w]+2])
        distD['2aaBlock_match_seq'].append(List2aaBlock_match_seq)



print(distD)

{'GeneName': ['ADNP', 'AHR', 'AHR', 'AHR', 'AHR', 'AHR', 'ALX1', 'ALX1', 'ALX3', 'ALX3', 'ALX4', 'ALX4', 'AR', 'AR', 'ARID5A', 'ARID5A', 'ARID5A', 'ARID5B', 'ARID5B', 'ARNT2', 'ARNTL', 'ARX', 'ASCL3', 'ATF2', 'ATF6', 'ATF6', 'ATF6', 'ATF6B', 'BACH1', 'BACH2', 'BACH2', 'BACH2', 'BBX', 'BCL11B', 'BCL11B', 'BCL6', 'BHLHE40', 'BHLHE41', 'BHLHE41', 'BNC1', 'BNC1', 'BNC1', 'BNC2', 'BSX', 'CASZ1', 'CDC5L', 'CDX1', 'CDX4', 'CEBPA', 'CIC', 'CREB3', 'CREB3L1', 'CREB3L1', 'CREB3L1', 'CREB3L2', 'CREB3L3', 'CREB5', 'CREM', 'CRX', 'CSRNP1', 'CSRNP3', 'CSRNP3', 'CSRNP3', 'CUX1', 'CUX2', 'DBP', 'DDIT3', 'DDIT3', 'DLX2', 'DLX3', 'DLX3', 'DLX5', 'DMBX1', 'DMRT1', 'DMRT1', 'DMRT3', 'DMRT3', 'DMRTA1', 'DMRTC2', 'E2F7', 'E2F7', 'E2F8', 'EBF1', 'EBF2', 'EBF3', 'EBF4', 'EGR1', 'EGR1', 'EGR1', 'EGR2', 'EGR2', 'EGR3', 'EGR4', 'EGR4', 'ELF4', 'ELK3', 'ELK3', 'ELK4', 'EMX1', 'EMX1', 'EOMES', 'EOMES', 'EPAS1', 'EPAS1', 'ERG', 'ERG', 'ESR1', 'ESR1', 'ESR2', 'ESX1', 'ETS1', 'ETS1', 'ETS1', 'ETS2', 'ETV1', 'ETV1', '

In [22]:
periodic_genes = [*set([distD['GeneName'] for a_dict in distD][0])]

print(len(periodic_genes))

641


In [23]:
new = pd.DataFrame.from_dict(distD)

new['PB_ID'] = new.groupby('GeneName').cumcount().add(1)
new['PB_ID'] = new.apply(lambda row: row['GeneName'] + '_PB_' + str(row['PB_ID']), axis = 1)
new

,GeneName,GeneID,TrnID,PepID,UniProtKB_ID,UniProtKB_Gene_Name,GeneFamily,spacer_type,len,FYW,...,mean,median,cv,in_IDR,IDR_num,IDR_match_seq,4aaBlock_match_seq,8aaBlock_match_seq,2aaBlock_match_seq_masked,PB_ID
0,ADNP,ENSG00000101126,ENST00000621696,ENSP00000483881,Q9H2P0,ADNP_HUMAN,Homeobox,4_10,"[31, 4, 5, 0, 6]",5,...,9.20,5.0,1.347563,True,2,EQLKWKNSSYGKVEGFWSKDQSQWKNAS,"[QLKWKNSS, NSSYGKVE, VEGFWSKD, EGFWSKDQ, QSQWK...","[GDREQLKWKNSSYGKV, LKWKNSSYGKVEGFWS, SYGKVEGFW...","[LXN, SXK, EXS, GXK, SXN]",ADNP_PB_1
1,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[10, 3, 0, 9]",4,...,5.50,6.0,0.871969,True,0,STAPFENNFFNESMNECRNWQDNT,"[TAPFENNF, ENNFFNES, NNFFNESM, CRNWQDNT]","[STSSTAPFENNFFNES, TAPFENNFFNESMNEC, APFENNFFN...","[AXN, NXN, NXE, RXD]",AHR_PB_1
2,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[26, 7, 8, 9]",4,...,12.50,8.5,0.722957,True,0,DVNSFAGGHPGLFQDSKNSDLYSIMKNLGIDFEDIR,"[VNSFAGGH, PGLFQDSK, SDLYSIMK, GIDFEDIR]","[QPQDVNSFAGGHPGLF, AGGHPGLFQDSKNSDL, DSKNSDLYS...","[NXG, GXD, DXI, IXD]",AHR_PB_2
3,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[10, 0, 3, 5]",4,...,4.50,4.0,0.934039,True,0,QNEKFFRNDFSGEVDFRDID,"[NEKFFRND, EKFFRNDF, RNDFSGEV, EVDFRDID]","[RHMQNEKFFRNDFSGE, HMQNEKFFRNDFSGEV, EKFFRNDFS...","[EXR, KXN, NXG, VXD]",AHR_PB_3
4,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[18, 4, 4, 8]",4,...,8.50,6.0,0.777421,True,0,TELDYPMGSFEPSPYPTTSSLEDFVTCL,"[ELDYPMGS, MGSFEPSP, PSPYPTTS, LEDFVTCL]","[SKCTELDYPMGSFEPS, LDYPMGSFEPSPYPTT, GSFEPSPYP...","[LXM, GXP, SXT, EXT]",AHR_PB_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,ZSCAN4,ENSG00000180532,ENST00000318203,ENSP00000321963,Q8NAM6,ZSCA4_HUMAN,zf-C2H2,4_10,"[16, 7, 6, 8]",4,...,9.25,7.5,0.494430,True,0,GISEFSRMVLNSFQDSNNSYARQELQRLYRIFH,"[ISEFSRMV, LNSFQDSN, NNSYARQE, QRLYRIFH]","[REEGISEFSRMVLNSF, SRMVLNSFQDSNNSYA, FQDSNNSYA...","[SXR, NXD, NXR, RXI]",ZSCAN4_PB_1
999,ZXDA,ENSG00000198205,ENST00000358697,ENSP00000351530,P98168,ZXDA_HUMAN,zf-C2H2,4_10,"[70, 7, 0, 7]",4,...,21.00,7.0,1.563472,True,1,AKAEWSVHPNSDFFGQEGETQFGFPN,"[KAEWSVHP, NSDFFGQE, SDFFGQEG, ETQFGFPN]","[LTPAKAEWSVHPNSDF, SVHPNSDFFGQEGETQ, VHPNSDFFG...","[AXV, SXG, DXQ, TXF]",ZXDA_PB_1
1000,ZXDA,ENSG00000198205,ENST00000358697,ENSP00000351530,P98168,ZXDA_HUMAN,zf-C2H2,11_20,"[71, 14, 14, 19]",4,...,29.50,16.5,0.941250,True,0,PAAAFAGTVTIHNQDLLLRFENGVLTLATPPPHAWEPGAAPAQQPR...,"[AAAFAGTV, LLRFENGV, PHAWEPGA, QAGFPQAA]","[APGPAAAFAGTVTIHN, NQDLLLRFENGVLTLA, ATPPPHAWE...","[AXG, LXN, HXP, AXQ]",ZXDA_PB_2
1001,ZXDB,ENSG00000198455,ENST00000374888,ENSP00000364023,P98169,ZXDB_HUMAN,zf-C2H2,4_10,"[70, 7, 0, 7]",4,...,21.00,7.0,1.563472,True,1,TKAEWNVHPDSDFFGQEGETQFGFPN,"[KAEWNVHP, DSDFFGQE, SDFFGQEG, ETQFGFPN]","[LTPTKAEWNVHPDSDF, NVHPDSDFFGQEGETQ, VHPDSDFFG...","[AXV, SXG, DXQ, TXF]",ZXDB_PB_1


In [24]:
new.to_csv('Periodic_blocks_HG38_TFs_0-3_4-10_11-20_21-30_threshold_4_06022023_.csv', index=False)

In [25]:
def filtering_df(x):
    spacers = x['len']
    if len(spacers) == 4 and 0 in spacers:
        return "YES"
    else: return "NO"

new["filter"] = new.apply(filtering_df , axis = 1)

new = new[new["filter"] == 'NO']
new = new.drop('filter', axis=1)

new

,GeneName,GeneID,TrnID,PepID,UniProtKB_ID,UniProtKB_Gene_Name,GeneFamily,spacer_type,len,FYW,...,mean,median,cv,in_IDR,IDR_num,IDR_match_seq,4aaBlock_match_seq,8aaBlock_match_seq,2aaBlock_match_seq_masked,PB_ID
0,ADNP,ENSG00000101126,ENST00000621696,ENSP00000483881,Q9H2P0,ADNP_HUMAN,Homeobox,4_10,"[31, 4, 5, 0, 6]",5,...,9.200000,5.0,1.347563,True,2,EQLKWKNSSYGKVEGFWSKDQSQWKNAS,"[QLKWKNSS, NSSYGKVE, VEGFWSKD, EGFWSKDQ, QSQWK...","[GDREQLKWKNSSYGKV, LKWKNSSYGKVEGFWS, SYGKVEGFW...","[LXN, SXK, EXS, GXK, SXN]",ADNP_PB_1
2,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[26, 7, 8, 9]",4,...,12.500000,8.5,0.722957,True,0,DVNSFAGGHPGLFQDSKNSDLYSIMKNLGIDFEDIR,"[VNSFAGGH, PGLFQDSK, SDLYSIMK, GIDFEDIR]","[QPQDVNSFAGGHPGLF, AGGHPGLFQDSKNSDL, DSKNSDLYS...","[NXG, GXD, DXI, IXD]",AHR_PB_2
4,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[18, 4, 4, 8]",4,...,8.500000,6.0,0.777421,True,0,TELDYPMGSFEPSPYPTTSSLEDFVTCL,"[ELDYPMGS, MGSFEPSP, PSPYPTTS, LEDFVTCL]","[SKCTELDYPMGSFEPS, LDYPMGSFEPSPYPTT, GSFEPSPYP...","[LXM, GXP, SXT, EXT]",AHR_PB_4
5,AHR,ENSG00000106546,ENST00000242057,ENSP00000242057,P35869,AHR_HUMAN,bHLH,4_10,"[15, 9, 3, 8]",4,...,8.750000,8.5,0.562792,True,0,GQMQYNPVLPGQQAFLNKFQNGVLNETYPAEL,"[QMQYNPVL, QQAFLNKF, LNKFQNGV, NETYPAEL]","[THVGQMQYNPVLPGQQ, VLPGQQAFLNKFQNGV, QQAFLNKFQ...","[MXP, QXN, NXN, EXA]",AHR_PB_5
7,ALX1,ENSG00000180318,ENST00000316824,ENSP00000315417,Q15699,ALX1_HUMAN,Homeobox,11_20,"[4, 11, 0, 18, 0, 11]",6,...,7.333333,7.5,0.982071,True,0,LSEKFALKSPPSKNSDFYMGAGGPLEHVMETLDNESFYSKASAGKC...,"[SEKFALKS, NSDFYMGA, SDFYMGAG, NESFYSKA, ESFYS...","[MEFLSEKFALKSPPSK, PPSKNSDFYMGAGGPL, PSKNSDFYM...","[EXL, SXM, DXG, EXS, SXK, QXP]",ALX1_PB_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,ZSCAN25,ENSG00000197037,ENST00000394152,ENSP00000377708,Q6NSZ9,ZSC25_HUMAN,zf-C2H2,4_10,"[27, 0, 9, 6, 4]",5,...,9.200000,6.0,1.138454,True,0,MAAGFFTAGSQGLGPFKDMALAFPEEEWRHVT,"[AAGFFTAG, AGFFTAGS, LGPFKDMA, ALAFPEEE, EEEWR...","[DQEMAAGFFTAGSQGL, QEMAAGFFTAGSQGLG, GSQGLGPFK...","[AXT, GXA, GXD, LXE, EXH]",ZSCAN25_PB_1
996,ZSCAN30,ENSG00000186814,ENST00000333206,ENSP00000329738,Q86W11,ZSC30_HUMAN,zf-C2H2,4_10,"[18, 5, 6, 4, 3]",5,...,7.200000,5.0,0.852782,True,0,EEENYVLDQDFGLQENPWSQEVFRQKFRQFS,"[EENYVLDQ, DQDFGLQE, ENPWSQEV, QEVFRQKF, RQKFR...","[VKVEEENYVLDQDFGL, NYVLDQDFGLQENPWS, FGLQENPWS...","[EXL, QXL, NXQ, EXQ, QXQ]",ZSCAN30_PB_1
998,ZSCAN4,ENSG00000180532,ENST00000318203,ENSP00000321963,Q8NAM6,ZSCA4_HUMAN,zf-C2H2,4_10,"[16, 7, 6, 8]",4,...,9.250000,7.5,0.494430,True,0,GISEFSRMVLNSFQDSNNSYARQELQRLYRIFH,"[ISEFSRMV, LNSFQDSN, NNSYARQE, QRLYRIFH]","[REEGISEFSRMVLNSF, SRMVLNSFQDSNNSYA, FQDSNNSYA...","[SXR, NXD, NXR, RXI]",ZSCAN4_PB_1
1000,ZXDA,ENSG00000198205,ENST00000358697,ENSP00000351530,P98168,ZXDA_HUMAN,zf-C2H2,11_20,"[71, 14, 14, 19]",4,...,29.500000,16.5,0.941250,True,0,PAAAFAGTVTIHNQDLLLRFENGVLTLATPPPHAWEPGAAPAQQPR...,"[AAAFAGTV, LLRFENGV, PHAWEPGA, QAGFPQAA]","[APGPAAAFAGTVTIHN, NQDLLLRFENGVLTLA, ATPPPHAWE...","[AXG, LXN, HXP, AXQ]",ZXDA_PB_2


In [26]:
new.to_csv('Periodic_blocks_HG38_TFs_0-3_4-10_11-20_21-30_threshold_4_06022023.csv', index=False)
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 1002
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   GeneName                   748 non-null    object 
 1   GeneID                     748 non-null    object 
 2   TrnID                      748 non-null    object 
 3   PepID                      748 non-null    object 
 4   UniProtKB_ID               748 non-null    object 
 5   UniProtKB_Gene_Name        748 non-null    object 
 6   GeneFamily                 748 non-null    object 
 7   spacer_type                748 non-null    object 
 8   len                        748 non-null    object 
 9   FYW                        748 non-null    int64  
 10  match                      748 non-null    object 
 11  coordinates                748 non-null    object 
 12  sd                         748 non-null    float64
 13  mean                       748 non-null    float6

In [27]:
periodic_genes = [*set(new['GeneName'].tolist())]

print(len(periodic_genes))

531


In [28]:
from Bio import SeqIO
from Bio import SeqRecord
from Bio import SeqFeature
zff = ['zf-C2H2', 'ZBTB', 'zf-GATA', 'zf-C2HC', 'zf-NF-X1', 'zf-BED']
new_fasta = []
new_fasta_block4 = []
new_fasta_block8 = []
new_fasta_block2 = []

for i, row in new.iterrows():
    if new['GeneFamily'][i] not in zff:
        seq1 = new['IDR_match_seq'][i]
        id_seq = new['PB_ID'][i]
        new_fasta.append('>%s\n%s' % (id_seq, seq1))
        for j in range(len(new['4aaBlock_match_seq'][i])):
            if len(new['4aaBlock_match_seq'][i][j]) > 7:
                id_seq2 = new['PB_ID'][i] + "_" + str(j)
                seq2 = new['4aaBlock_match_seq'][i][j]
                new_fasta_block4.append('>%s\n%s' % (id_seq2, seq2))
        for j in range(len(new['8aaBlock_match_seq'][i])):
            if len(new['8aaBlock_match_seq'][i][j]) > 7:
                id_seq3 = new['PB_ID'][i] + "_" + str(j)
                seq3 = new['8aaBlock_match_seq'][i][j]
                new_fasta_block8.append('>%s\n%s' % (id_seq3, seq3))
        for j in range(len(new['2aaBlock_match_seq'][i])):
            if len(new['2aaBlock_match_seq'][i][j]) > 3:
                id_seq4 = new['PB_ID'][i] + "_" + str(j)
                seq4 = new['2aaBlock_match_seq'][i][j]
                new_fasta_block2.append('>%s\n%s' % (id_seq4, seq4))


with open('TF_PB_sequences.fasta', 'w') as fasta_file:  # Will close handle cleanly
    fasta_file.write('\n'.join(new_fasta))

with open('TF_PB_4aablock_sequences.fasta', 'w') as fasta_file2:  # Will close handle cleanly
    fasta_file2.write('\n'.join(new_fasta_block4))

with open('TF_PB_8aablock_sequences.fasta', 'w') as fasta_file3:  # Will close handle cleanly
    fasta_file3.write('\n'.join(new_fasta_block8))

with open('TF_PB_2aablock_sequences.fasta', 'w') as fasta_file4:  # Will close handle cleanly
    fasta_file4.write('\n'.join(new_fasta_block2))


In [23]:
Sanborn_high  = pd.read_csv('TaD_database/human_TF_ADs_high-strength.csv',low_memory=False)
Sanborn_medium = pd.read_csv('TaD_database/human_TF_ADs_medium-strength.csv',low_memory=False)
Staller = pd.read_csv('TaD_database/predictions_Staller.csv',low_memory=False)
Erijman = pd.read_csv('TaD_database/Erijman_prediction_results.csv',low_memory=False)

Sanborn_high_genes = [*set(Sanborn_high["prot_name"].tolist())]
Sanborn_medium_genes = [*set(Sanborn_medium["prot_name"].tolist())]
Staller_genes = [*set(Staller["Uniprot_name"].tolist())]
Erijman_genes = [*set(Erijman["prot_name"].tolist())]


Sanborn_high = Sanborn_high[['prot_name','start', 'stop']]
Sanborn_medium = Sanborn_medium[['prot_name','start', 'stop']]
Staller = Staller[['Uniprot_name','Start', 'End']]
Erijman = Erijman[['prot_name','start', 'stop']]

Sanborn_high = Sanborn_high.groupby('prot_name').apply(lambda x: [list(x['start']), list(x['stop'])]).apply(pd.Series).reset_index()
Sanborn_medium = Sanborn_medium.groupby('prot_name').apply(lambda x: [list(x['start']), list(x['stop'])]).apply(pd.Series).reset_index()
Staller = Staller.groupby('Uniprot_name').apply(lambda x: [list(x['Start']), list(x['End'])]).apply(pd.Series).reset_index()
Erijman = Erijman.groupby('prot_name').apply(lambda x: [list(x['start']), list(x['stop'])]).apply(pd.Series).reset_index()

Sanborn_high.columns = ['prot_name','start', 'stop']
Sanborn_medium.columns = ['prot_name','start', 'stop']
Staller.columns = ['prot_name','start', 'stop']
Erijman.columns = ['prot_name','start', 'stop']

#Sanborn_high.head()
#Sanborn_medium.head()
Erijman.head()

,prot_name,start,stop
0,ADNP2_HUMAN,[812],[819]
1,ADNP_HUMAN,[36],[54]
2,AHRR_HUMAN,[163],[169]
3,AHR_HUMAN,[562],[588]
4,AIRE_HUMAN,[517],[530]


In [24]:
UniProtKB_Gene_Name in Sanborn_medium_genes

False

In [25]:
def flatten(listOfLists):
    return chain.from_iterable(listOfLists)

Alerasool =["RBPJ", "ATXN7L3", "CITED2", "CITED1", "ATF6", "KLF6", "DDIT3", "ATF6B", "ZXDC", "SERTAD2", "C3orf62", "RELB", "FAM90A1", "NPAS4", "YAF2", "LIN9", "KLF7", "KLF15", "FAM22F", "SPDYE4", "C11orf74", "BRD8", "ELF4", "VP64"]


periodic = dict()
periodic['GeneName']=list()
periodic['GeneID'] = list()
periodic['TrnID'] = list()
periodic['PepID'] = list()
periodic['UniProtKB_ID'] = list()
periodic['UniProtKB_Gene_Name'] = list()
periodic['GeneFamily']=list()
periodic['FYW']=list()
periodic['Score']=list()
periodic['num_block']=list()
periodic['aa_blocks']=list()
periodic['IDR_num']=list()
periodic['seq_periodic']=list()
periodic['is_ZF']=list()
periodic['in_Alerasool']=list()
periodic['in_Sanborn_high']=list()
periodic['in_Sanborn_medium']=list()
periodic['in_Staller']=list()
periodic['in_Erijman']=list()
periodic['is_Alerasool_in_IDR']=list()
periodic['is_Sanborn_high_in_IDR']=list()
periodic['is_Sanborn_medium_in_IDR']=list()
periodic['is_Staller_in_IDR']=list()
periodic['is_Erijman_in_IDR']=list()


for gene in periodic_genes:

    #prepare variables for gene
    temp = new[ (new["GeneName"] == gene)]
    genefamily = [*set(temp["GeneFamily"].values.tolist())][0]

    if gene in Alerasool:
        in_Alerasool = "Yes"
        is_Alerasool_in_IDR = "Yes"
    else:
        in_Alerasool = "No"
        is_Alerasool_in_IDR = "No"

    if  "zf" in genefamily:
        is_ZF = "Yes"
    else:
        is_ZF = "No"

    geneid = [*set(temp["GeneID"].values.tolist())][0]
    trnID = [*set(temp["TrnID"].values.tolist())][0]
    pepID = [*set(temp["PepID"].values.tolist())][0]
    UniProtKB_ID = [*set(temp["UniProtKB_ID"].values.tolist())][0]
    UniProtKB_Gene_Name = [*set(temp["UniProtKB_Gene_Name"].values.tolist())][0]

    ################################################################
    ## Sanborn_high
    ################################################################
    if UniProtKB_Gene_Name in Sanborn_high_genes:

        in_Sanborn_high = True

        Sanborn_high_temp = Sanborn_high[ (Sanborn_high["prot_name"] == UniProtKB_Gene_Name) ]

        idr_temp = df_test[ (df_test["GeneName"] == gene)]

        IDR_coordinates = []

        for i, row in idr_temp.iterrows():
            idr_s = idr_temp['IDR_START'][i]
            idr_e = idr_temp['IDR_END'][i]

            for n in range(len(idr_s)):
                IDR_coordinates.append(pd.Interval(int(idr_s[n]), int(idr_e[n]), closed='both'))

        arrayIDR = pd.arrays.IntervalArray(IDR_coordinates)

        # Check if there are any tads from Sanborn_high

        Sanborn_high_coordinates = []

        for i, row in Sanborn_high_temp.iterrows():
            Sanborn_high_s = Sanborn_high_temp['start'][i]
            Sanborn_high_e = Sanborn_high_temp['stop'][i]

            for n in range(len(Sanborn_high_s)):
                Sanborn_high_coordinates.append(pd.Interval(int(Sanborn_high_s[n]), int(Sanborn_high_e[n]), closed='both'))

        array_Sanborn_high = pd.arrays.IntervalArray(Sanborn_high_coordinates)

        testSanborn_high = []

        for i in range(len(array_Sanborn_high)):
            test = array_Sanborn_high[i]
            testSanborn_high.append(arrayIDR.overlaps(test))



        testSanborn_high2 = list(itertools.chain(*testSanborn_high))
        is_Sanborn_high_in_IDR = True in testSanborn_high2

    else:
        in_Sanborn_high = False
        is_Sanborn_high_in_IDR = False

    ################################################################
    ## Sanborn_medium
    ################################################################

    if UniProtKB_Gene_Name in Sanborn_medium_genes:

        in_Sanborn_medium = True

        Sanborn_medium_temp = Sanborn_medium[ (Sanborn_medium["prot_name"] == UniProtKB_Gene_Name) ]

        idr_temp = df_test[ (df_test["GeneName"] == gene)]

        IDR_coordinates = []

        for i, row in idr_temp.iterrows():
            idr_s = idr_temp['IDR_START'][i]
            idr_e = idr_temp['IDR_END'][i]

            for n in range(len(idr_s)):
                IDR_coordinates.append(pd.Interval(int(idr_s[n]), int(idr_e[n]), closed='both'))

        arrayIDR = pd.arrays.IntervalArray(IDR_coordinates)

        # Check if there are any tads from Sanborn_high

        Sanborn_medium_coordinates = []

        for i, row in Sanborn_medium_temp.iterrows():
            Sanborn_medium_s = Sanborn_medium_temp['start'][i]
            Sanborn_medium_e = Sanborn_medium_temp['stop'][i]

            for n in range(len(Sanborn_medium_s)):
                Sanborn_medium_coordinates.append(pd.Interval(int(Sanborn_medium_s[n]), int(Sanborn_medium_e[n]), closed='both'))

        array_Sanborn_medium = pd.arrays.IntervalArray(Sanborn_medium_coordinates)

        testSanborn_medium = []

        for i in range(len(array_Sanborn_medium)):
            test = array_Sanborn_medium[i]
            testSanborn_medium.append(arrayIDR.overlaps(test))



        testSanborn_medium2 = list(itertools.chain(*testSanborn_medium))
        is_Sanborn_medium_in_IDR = True in testSanborn_medium2

    else:
        in_Sanborn_medium = False
        is_Sanborn_medium_in_IDR = False

    ################################################################
    ## Staller
    ################################################################

    if UniProtKB_Gene_Name in Staller_genes:

        in_Staller = True

        Staller_temp = Staller[ (Staller["prot_name"] == UniProtKB_Gene_Name) ]

        idr_temp = df_test[ (df_test["GeneName"] == gene)]

        IDR_coordinates = []

        for i, row in idr_temp.iterrows():
            idr_s = idr_temp['IDR_START'][i]
            idr_e = idr_temp['IDR_END'][i]

            for n in range(len(idr_s)):
                IDR_coordinates.append(pd.Interval(int(idr_s[n]), int(idr_e[n]), closed='both'))

        arrayIDR = pd.arrays.IntervalArray(IDR_coordinates)

        # Check if there are any tads from Staller

        Staller_coordinates = []

        for i, row in Staller_temp.iterrows():
            Staller_s = Staller_temp['start'][i]
            Staller_e = Staller_temp['stop'][i]

            for n in range(len(Staller_s)):
                Staller_coordinates.append(pd.Interval(int(Staller_s[n]), int(Staller_e[n]), closed='both'))

        array_Staller = pd.arrays.IntervalArray(Staller_coordinates)

        testStaller = []

        for i in range(len(array_Staller)):
            test = array_Staller[i]
            testStaller.append(arrayIDR.overlaps(test))



        testStaller2 = list(itertools.chain(*testStaller))
        is_Staller_in_IDR = True in testStaller2

    else:
        in_Staller = False
        is_Staller_in_IDR = False

    ################################################################
    ## Erijman
    ################################################################

    if UniProtKB_Gene_Name in Erijman_genes:

        in_Erijman = True

        Erijman_temp = Erijman[ (Erijman["prot_name"] == UniProtKB_Gene_Name) ]

        idr_temp = df_test[ (df_test["GeneName"] == gene)]

        IDR_coordinates = []

        for i, row in idr_temp.iterrows():
            idr_s = idr_temp['IDR_START'][i]
            idr_e = idr_temp['IDR_END'][i]

            for n in range(len(idr_s)):
                IDR_coordinates.append(pd.Interval(int(idr_s[n]), int(idr_e[n]), closed='both'))

        arrayIDR = pd.arrays.IntervalArray(IDR_coordinates)

        # Check if there are any tads from Erijman

        Erijman_coordinates = []

        for i, row in Erijman_temp.iterrows():
            Erijman_s = Erijman_temp['start'][i]
            Erijman_e = Erijman_temp['stop'][i]

            for n in range(len(Erijman_s)):
                Erijman_coordinates.append(pd.Interval(int(Erijman_s[n]), int(Erijman_e[n]), closed='both'))

        array_Erijman = pd.arrays.IntervalArray(Erijman_coordinates)

        testErijman = []

        for i in range(len(array_Erijman)):
            test = array_Erijman[i]
            testErijman.append(arrayIDR.overlaps(test))



        testErijman2 = list(itertools.chain(*testErijman))
        is_Erijman_in_IDR = True in testErijman2

    else:
        in_Erijman = False
        is_Erijman_in_IDR = False

    #set sequence that is periodic
    numb_IDR = [*set(temp['IDR_num'].values.tolist())]

    score_perIDR = dict()
    score_perIDR['IDR_num'] = list()
    score_perIDR['IDR_score'] = list()
    score_perIDR['IDR_aa_blocks'] = list()
    score_perIDR['FYW'] = list()
    score_perIDR['num_block'] = list()
    score_perIDR['seq_periodic'] = list()

    for i in numb_IDR:

        IDR_aa_blocks = []

        temp2 = temp[ (temp["IDR_num"] == i)]
        IDR_num = i
        num_block = len(temp2.index)

        tempg = temp2.groupby('IDR_num').agg({'coordinates':lambda x: sorted(list(flatten(list(x))))})
        tempseq = df_test[ (df_test['GeneName'] == gene)]['sequence'].iloc[0]
        seq_periodic = list()
#        aromatic_periodic = list()

        for i in range(len(tempg)):
            start = tempg.iloc[i][0][0]
            end = tempg.iloc[i][0][-1]
            seq_periodic.append(tempseq[start:end])

        # for i in range(len(seq_periodic)):
        #     aromatic_periodic.append(seq_periodic[i].count("F") + seq_periodic[i].count("Y") + seq_periodic[i].count("W"))


        aa_blocks = np.array(temp2['FYW'].values.tolist())
        total_periodic_AA = sum(temp2['FYW'].values.tolist())
        weights= aa_blocks/total_periodic_AA

        dist_weight = list()
        dist_weight_temp = temp2['spacer_type'].values.tolist()
        for i in range(len(dist_weight_temp)):
            if dist_weight_temp[i] == "0_3":
                dist_weight.append(1)

            if dist_weight_temp[i] == "4_10":
                dist_weight.append(1)

            if dist_weight_temp[i] == "11_20":
                dist_weight.append(1.1)

            if dist_weight_temp[i] == "21_30":
                dist_weight.append(1.2)
        dist_weight = pd.Series(dist_weight)
        aa_blocks = pd.Series(aa_blocks)
        score = weights.dot(aa_blocks.mul(dist_weight))*num_block
        aa_blocks = aa_blocks.tolist()
        IDR_aa_blocks.append(aa_blocks)

        score_perIDR['IDR_num'].append(IDR_num)
        score_perIDR['IDR_score'].append(score)
        score_perIDR['IDR_aa_blocks'].append(IDR_aa_blocks)
        score_perIDR['FYW'].append(total_periodic_AA)
        score_perIDR['num_block'].append(num_block)
        score_perIDR['seq_periodic'].append(seq_periodic)


    for i in range(len(score_perIDR['IDR_num'])):
        score = score_perIDR['IDR_score'][i]
        num_block = score_perIDR['num_block'][i]
        total_periodic_AA = score_perIDR['FYW'][i]
        aa_blocks = list(flatten(score_perIDR['IDR_aa_blocks'][i]))
        IDR_num = score_perIDR['IDR_num'][i] + 1
        seq_periodic = score_perIDR['seq_periodic'][i]

        periodic['GeneName'].append(gene)
        periodic['GeneID'].append(geneid)
        periodic['TrnID'].append(trnID)
        periodic['PepID'].append(pepID)
        periodic['UniProtKB_ID'].append(UniProtKB_ID)
        periodic['UniProtKB_Gene_Name'].append(UniProtKB_Gene_Name)
        periodic['GeneFamily'].append(genefamily)
        periodic['Score'].append(round(score, 2))
        periodic['num_block'].append(num_block)
        periodic['FYW'].append(total_periodic_AA)
        periodic['aa_blocks'].append(aa_blocks)
        periodic['IDR_num'].append(IDR_num)
        periodic['seq_periodic'].append(seq_periodic)
        periodic['is_ZF'].append(is_ZF)
        #annotation of TaD data
        periodic['in_Alerasool'].append(in_Alerasool)
        periodic['in_Sanborn_high'].append(in_Sanborn_high)
        periodic['in_Sanborn_medium'].append(in_Sanborn_medium)
        periodic['in_Staller'].append(in_Staller)
        periodic['in_Erijman'].append(in_Erijman)
        periodic['is_Alerasool_in_IDR'].append(is_Alerasool_in_IDR)
        periodic['is_Sanborn_high_in_IDR'].append(is_Sanborn_high_in_IDR)
        periodic['is_Sanborn_medium_in_IDR'].append(is_Sanborn_medium_in_IDR)
        periodic['is_Staller_in_IDR'].append(is_Staller_in_IDR)
        periodic['is_Erijman_in_IDR'].append(is_Erijman_in_IDR)

print(periodic)

{'GeneName': ['HOXB3', 'ESR2', 'ZNF761', 'ZNF763', 'THAP9', 'TBX22', 'NKX2-5', 'FOXC2', 'HIVEP1', 'ZEB2', 'TEF', 'ZFPM2', 'ZNF431', 'MLXIPL', 'ZNF160', 'PAX3', 'ZNF415', 'ZNF680', 'ZNF620', 'ZNF711', 'NFIX', 'ZNF396', 'ZNF396', 'FOSL1', 'FLI1', 'FLI1', 'TBX15', 'NKX1-1', 'ZNF212', 'ETS2', 'MYRF', 'MYRF', 'ZBTB44', 'ZNF566', 'HIF3A', 'SOX6', 'ZNF441', 'CREB3L3', 'HOXC6', 'ZNF554', 'ZNF317', 'ZNF429', 'ZNF142', 'NKX2-3', 'NKX2-3', 'FOXD4L4', 'CREB5', 'PRDM9', 'TBX21', 'TBX21', 'ZNF479', 'SMAD1', 'ZNF551', 'TLX1', 'AR', 'HIVEP3', 'HIVEP3', 'ZNF460', 'ZNF829', 'ZNF184', 'MGA', 'MGA', 'MYBL1', 'RLF', 'RLF', 'HOXD4', 'PRDM15', 'HOXA3', 'KLF4', 'ZNF85', 'RFX2', 'MYOG', 'SIM1', 'ZNF467', 'ZNF606', 'CREB3L2', 'ZSCAN4', 'ZNF493', 'NR3C1', 'ZNF766', 'CDX4', 'ELF4', 'FOXN3', 'RFX4', 'ZNF445', 'ZNF510', 'HOXB4', 'ZNF768', 'ZNF639', 'ZNF470', 'SP3', 'GCM1', 'IKZF4', 'ZBED1', 'ZNF404', 'ZNF565', 'ZNF225', 'CREB3', 'NKX2-6', 'ZNF549', 'ZNF625', 'ZNF248', 'IRX2', 'TFAP2A', 'ZNF555', 'ZNF75A', 'CASZ1', 

In [26]:
periodic_df = pd.DataFrame.from_dict(periodic)
periodic_df = periodic_df.sort_values('Score', ascending=False)
periodic_df.head()

,GeneName,GeneID,TrnID,PepID,UniProtKB_ID,UniProtKB_Gene_Name,GeneFamily,FYW,Score,num_block,...,in_Alerasool,in_Sanborn_high,in_Sanborn_medium,in_Staller,in_Erijman,is_Alerasool_in_IDR,is_Sanborn_high_in_IDR,is_Sanborn_medium_in_IDR,is_Staller_in_IDR,is_Erijman_in_IDR
240,NPAS4,ENSG00000174576,ENST00000311034,ENSP00000311196,Q8IUM7,NPAS4_HUMAN,bHLH,21,26.42,4,...,Yes,True,False,True,True,Yes,True,False,True,True
182,HSF5,ENSG00000176160,ENST00000323777,ENSP00000313243,Q4G112,HSF5_HUMAN,HSF,22,23.82,5,...,No,False,True,False,False,No,False,True,False,False
213,NFAT5,ENSG00000102908,ENST00000349945,ENSP00000338806,O94916,NFAT5_HUMAN,RHD,19,23.49,4,...,No,True,False,False,False,No,True,False,False,False
419,ZNF214,ENSG00000149050,ENST00000278314,ENSP00000278314,Q9UL59,ZN214_HUMAN,zf-C2H2,20,22.40,4,...,No,False,False,False,True,No,False,False,False,True
275,HOXB6,ENSG00000108511,ENST00000225648,ENSP00000225648,P17509,HXB6_HUMAN,Homeobox,17,20.18,2,...,No,False,False,False,False,No,False,False,False,False


In [27]:
periodic_df.to_csv('Periodic_Genes_HG38_TFs_0-3_4-10_11-20_21-30_scored_threshold_4_23012023.csv', index=False)